In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
all_features = [
        'PassengerId', 
        'Name', 
        'Ticket', 
        'Fare', 
        'SibSp', 
        'Parch', 
        'Age', 
        'Cabin', 
        'Embarked',
        'Pclass',
        'Sex',
           ]
used_features = [
#        'PassengerId', 
#        'Name', 
#        'Ticket', 
#        'Fare', 
        'SibSp', 
        'Parch', 
        'Age', 
        'Cabin', 
        'Embarked',
        'Pclass',
        'Sex',
           ]

In [4]:
def feature_to_int(row):
    return row.replace(to_replace=row.unique(), value=np.arange(len(row.unique())))

def feature_to_int_sorted(row):
    return row.replace(to_replace=row.unique(), value=np.arange(len(row.unique())))

def feature_to_normalized_float(row):
    sorted_int = row.replace(to_replace=row.unique(), value=np.arange(len(row.unique())))
    return (sorted_int - np.min(sorted_int))/np.ptp(sorted_int) 

def exclude(feat, excluded):
    for elem in excluded:
        feat.remove(elem)
    return feat

def process_dataset(df):
    # cabin to number
#    df['Cabin'] = feature_to_int(df['Cabin'])
    #embarked to number
#    df['Embarked'] = feature_to_int(df['Embarked'])
    #sex to number
#    df['Sex'] = feature_to_int(df['Sex'])
#    df['Ticket'] = feature_to_int(df['Ticket'])
    
#    df['Age'] = feature_to_int_sorted(df['Age'])
#    df['Fare'] = feature_to_int_sorted(df['Fare'])

    # comment out important features
    df = df.drop(columns=exclude(all_features.copy(),used_features))
    
    for col in df:
        df[col] = feature_to_normalized_float(df[col])
    
    return df

In [5]:
train_data = pd.read_csv('data/train.csv')
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
processed_train_data = process_dataset(train_data)
x_train, y_train = processed_train_data.drop(columns=['Survived']), processed_train_data['Survived']
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0.5,1.0,0.011364,0.000000,0.0,0.006803,0.333333
2,0.0,1.0,0.022727,0.166667,0.0,0.000000,0.000000
3,0.5,1.0,0.034091,0.000000,0.0,0.013605,0.000000
4,0.0,0.0,0.034091,0.166667,0.0,0.000000,0.000000


In [7]:
dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))

In [8]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. 0. 0. 0. 0.], Target: 0.0
Features: [0.5        1.         0.01136364 0.         0.         0.00680272
 0.33333333], Target: 1.0
Features: [0.         1.         0.02272727 0.16666667 0.         0.
 0.        ], Target: 1.0
Features: [0.5        1.         0.03409091 0.         0.         0.01360544
 0.        ], Target: 1.0
Features: [0.         0.         0.03409091 0.16666667 0.         0.
 0.        ], Target: 0.0


In [9]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
      ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
tf_train_dataset = dataset.shuffle(len(x_train)).batch(1)
model = get_compiled_model()

In [11]:
hist = model.fit(tf_train_dataset, epochs=9)
hist.history['accuracy'][-1]

Epoch 1/9

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

891/891 [==============================] - 1s 2ms/step - loss: 0.5744 - accuracy: 0.7329
Epoch 2/9
891/891 [==============================] - 1s 2ms/step - loss: 0.4907 - accuracy: 0.7969
Epoch 3/9
891/891 [==============================] - 2s 2ms/step - loss: 0.4872 - accuracy: 0.7901
Epoch 4/9
891/891 [==============================] - 1s 2ms/step - loss: 0.4717 - accuracy: 0.7980
Epoch 5/9
891/891 [==============================] - 1s 2ms/step - loss: 0.4770 - accuracy: 0.8002
Epoch 6/9
891

0.8125701546669006

In [12]:
test_data = pd.read_csv('data/test.csv')
processed_test_data = process_dataset(test_data)
processed_test_data

,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,0.0,1.0,0.012658,0.166667,0.000000,0.0,0.5
2,0.5,0.0,0.025316,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.037975,0.000000,0.000000,0.0,0.5
4,0.0,1.0,0.050633,0.166667,0.142857,0.0,0.5
...,...,...,...,...,...,...,...
413,0.0,0.0,0.126582,0.000000,0.000000,0.0,0.5
414,1.0,1.0,0.329114,0.000000,0.000000,1.0,1.0
415,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.5
416,0.0,0.0,0.126582,0.000000,0.000000,0.0,0.5


In [13]:
arr = model.predict(processed_test_data.values)
result = test_data['PassengerId'].to_frame() #.insert(1, 'Survived', arr.round())

In [14]:
result.insert(1, 'Survived', arr.round().astype(int))

In [15]:
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [16]:
result.to_csv('result.csv', encoding='utf-8', index=False)